In [224]:
%matplotlib inline
import pandas as pd
import statsmodels.formula.api as sm

In [225]:
rent_df = pd.read_excel('../../data/rent.xlsx')

In [226]:
rent_df['is_euro'] = rent_df['is_cosmetic'] == 0
rent_df['is_euro'] = rent_df['is_euro'].astype(int)
result = sm.ols(formula="rent ~ m_to_metro + is_euro", data=rent_df).fit()

In [227]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   rent   R-squared:                       0.368
Model:                            OLS   Adj. R-squared:                  0.359
Method:                 Least Squares   F-statistic:                     38.49
Date:                Thu, 19 Nov 2020   Prob (F-statistic):           6.76e-14
Time:                        07:13:00   Log-Likelihood:                -1367.6
No. Observations:                 135   AIC:                             2741.
Df Residuals:                     132   BIC:                             2750.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3.686e+04   1230.419     29.955      0.000    3.44e+04    3.93e+04
m_to_metro  -281.2514     99.122     -2.837      0.005    -477.324     -85.179
is_euro     8861.1200   1132.613      7.824      0.000    6620.700    1.11e+04
==============================================================================
Omnibus:                       14.076   Durbin-Watson:                   1.150
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               15.867
Skew:                           0.693   Prob(JB):                     0.000359
Kurtosis:                       3.948   Cond. No.                         30.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [228]:
copy = rent_df.copy()
copy = copy.append(dict(id=0, is_euro=0, m_to_metro=15), ignore_index=True)

copy['rent'] = 0

In [229]:
copy['rent'] = result.predict(copy)

In [230]:
new_df = rent_df.where(rent_df['is_cosmetic'] == 1).dropna()

In [231]:
only_cosmetic_reg = sm.ols(formula="rent ~ m_to_metro", data=new_df).fit()
only_cosmetic_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   rent   R-squared:                       0.063
Model:                            OLS   Adj. R-squared:                  0.052
Method:                 Least Squares   F-statistic:                     5.909
Date:                Thu, 19 Nov 2020   Prob (F-statistic):             0.0171
Time:                        07:13:00   Log-Likelihood:                -896.73
No. Observations:                  90   AIC:                             1797.
Df Residuals:                      88   BIC:                             1802.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3.629e+04   1129.913     32.118      0.000     3.4e+04    3.85e+04
m_to_metro  -227.5770     93.619     -2.431      0.017    -413.626     -41.528
==============================================================================
Omnibus:                       12.634   Durbin-Watson:                   0.942
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               13.398
Skew:                           0.839   Prob(JB):                      0.00123
Kurtosis:                       3.869   Cond. No.                         25.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [232]:
new_df = new_df.append(dict(m_to_metro=15), ignore_index=True)
new_df['pred_rent'] = only_cosmetic_reg.predict(new_df)

In [233]:
subject = new_df.iloc[90]['pred_rent']

In [234]:
new_df['proximity_adjustment'] = 1 - subject / new_df['pred_rent']

In [235]:
new_df.to_excel('../../data/prox_adjustment.xlsx')